# Plots for Near-Optimal Analysis
***

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from vresutils.snakemake import MockSnakemake, Dict

In [ ]:
from plotting import collection as clt

In [ ]:
plt.style.use('bmh')
%matplotlib inline

## Preliminaries

In [ ]:
fig_folder = "../graphics/"

In [ ]:
snakemake = MockSnakemake(
    input=Dict(
        investments="results/summaries/investments.csv",
        energy="results/summaries/energy.csv",
        storage_capacity="results/summaries/storage_capacity.csv",
        generation_capacity="results/summaries/generation_capacity.csv",
        line_capacity="results/summaries/line_capacity.csv",
        link_capacity="results/summaries/link_capacity.csv",
        line_volume="results/summaries/line_volume.csv",
        link_volume="results/summaries/link_volume.csv",
        line_energy_balance="results/summaries/line_energy_balance.csv",
        link_energy_balance="results/summaries/link_energy_balance.csv",
        ginis="results/summaries/gini.csv",
        network="results/networks/elec_s_100_ec_lcopt_2H.nc",
        network_tmin1="results/networks/elec_s_100_ec_lcopt_2H_tol0.01_cat-totals_obj-Transmission++min.nc",
        network_tmin5="results/networks/elec_s_100_ec_lcopt_2H_tol0.05_cat-totals_obj-Transmission++min.nc",
        network_tmin10="results/networks/elec_s_100_ec_lcopt_2H_tol0.1_cat-totals_obj-Transmission++min.nc",
        shapes="pypsa-eur/resources/regions_onshore_elec_s_100.geojson"
    )
)

## Read Data

In [ ]:
header = [i for i in range(7)]
investments = pd.read_csv(snakemake.input.investments, index_col=0, header=header)
energy=pd.read_csv(snakemake.input.energy, index_col=[0,1], header=header)
storage_capacity=pd.read_csv(snakemake.input.storage_capacity, index_col=[0,1], header=header)
generation_capacity=pd.read_csv(snakemake.input.generation_capacity, index_col=[0,1], header=header)
line_capacity=pd.read_csv(snakemake.input.line_capacity, index_col=0, header=header)
link_capacity=pd.read_csv(snakemake.input.link_capacity, index_col=0, header=header)
line_volume=pd.read_csv(snakemake.input.line_volume, index_col=0, header=header)
link_volume=pd.read_csv(snakemake.input.link_volume, index_col=0, header=header)
line_energy_balance=pd.read_csv(snakemake.input.line_energy_balance, index_col=0, header=header)
link_energy_balance=pd.read_csv(snakemake.input.link_energy_balance, index_col=0, header=header);

In [ ]:
def adjust_column_names(df):
    new_column_names = []
    for col in df.columns.tolist():
        col = list(col)  # tuple does not allow item assignment
        if col[-2] == "all":
            col[-2] = "-".join([col[-2], col[-3]])
        new_column_names.append(tuple(col))

    df.columns = pd.MultiIndex.from_tuples(new_column_names, names=df.columns.names)

In [ ]:
for df in [
    investments,
    line_volume,
    link_volume,
    generation_capacity,
    storage_capacity,
    energy,
]:
    adjust_column_names(df)

In [ ]:
def read_gini(gini_fn, ln_vol, lk_vol):
    tvol = (ln_vol.sum()/1e3 + lk_vol.sum()/1e3).reset_index()
    gini = pd.read_csv(gini_fn, index_col=0, header=header).T.reset_index()
    gini = pd.concat([gini,tvol[0]], axis=1)
    gini = gini.rename(columns={0: 'tvol'})
    return gini.loc[gini.variable_name=='all']

In [ ]:
gini = read_gini(snakemake.input.ginis, line_volume, link_volume)

In [ ]:
n = pypsa.Network(snakemake.input.network)
n_tmin1 = pypsa.Network(snakemake.input.network_tmin1)
n_tmin5 = pypsa.Network(snakemake.input.network_tmin5)
n_tmin10 = pypsa.Network(snakemake.input.network_tmin10)

In [ ]:
shapes = gpd.read_file(snakemake.input.shapes)

## Execute Plotting Functions

In [ ]:
clt.plot_gini(gini, n)

In [ ]:
lorentz_networks = {"Optimum": n, "Min TVOL 1%": n_tmin1, "Min TVOL 5%": n_tmin5, "Min TVOL 10%": n_tmin10}
clt.plot_lorentz(lorentz_networks)

In [ ]:
clt.plot_plain_boxplot(generation_capacity)

In [ ]:
clt.plot_curated_boxplots(generation_capacity)

In [ ]:
clt.plot_capacity_correlation(generation_capacity, storage_capacity, line_volume, link_volume)

In [ ]:
clt.plot_energy_correlation(energy)

In [ ]:
clt.plot_violins(generation_capacity, storage_capacity)

In [ ]:
clt.plot_energy_pie(n)

In [ ]:
clt.plot_cost_bar(n, order=True, relative=False)

In [ ]:
clt.plot_cost_bar(n, order=True, relative=True)

In [ ]:
clt.plot_dominance(n, n_tmin1, n_tmin5, n_tmin10, shapes=shapes, attr='energy')

In [ ]:
clt.plot_dominance(n, n_tmin1, n_tmin5, n_tmin10, shapes=shapes, attr='p_nom_opt')

In [ ]:
clt.plot_bar_collection(generation_capacity, storage_capacity)

In [ ]:
clt.plot_space(n, investments, line_volume, link_volume, compact=True)

In [ ]:
clt.plot_capacity_bar(n)

In [ ]:
for tol in [0.005,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]:
    fn = f"../results/networks/elec_s_100_ec_lcopt_2H_tol{tol}_cat-totals_obj-transmission++min.nc"
    n_tol = pypsa.Network(fn)
    clt.plot_network(n_tol, fn=fig_folder+f"map-tmin-{tol}.pdf")
    del n_tol

## Some Misc Checks

In [ ]:
(n.lines.s_nom_opt * n.lines.length / 1e6).sum()

In [ ]:
(n.links.p_nom_opt * n.links.length / 1e6).sum()

In [ ]:
n.storage_units.groupby('carrier').sum().p_nom_opt / 1e3

In [ ]:
n.generators.groupby('carrier').sum().p_nom_opt / 1e3 # GW

In [ ]:
n.generators['energy'] = n.generators_t.p.multiply(n.snapshot_weightings, axis=0).sum()
n.generators.groupby('carrier').sum().energy / 1e6 # TWh

In [ ]:
c = aggregate_costs(n)
c = c.groupby(level=2).sum() / 1e9 # bn Euro
c

In [ ]:
load = n.loads_t.p_set.multiply(n.snapshot_weightings, axis=0).sum().sum() / 1e6 #TWh
load

In [ ]:
c.sum()

In [ ]:
c.sum() / load * 1e3 # EUR/MWh

In [ ]:
net.global_constraints.mu

In [ ]:
get_system_transmission_volume(n, '_opt')

In [ ]:
n.generators.groupby("carrier").sum().p_nom_max / 1e3